In [7]:
# import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import os
import sys 

In [21]:
# import transactions dataset
df = pd.read_csv('transactions.csv')

# import auxiliary datasets (customer list, demographics, addresses)
df_customer_list = pd.read_csv('new_customer_list.csv')
df_demographics = pd.read_csv('customer_demographics.csv')
df_addresses = pd.read_csv('customer_addresses.csv')

       transaction_id  product_id  customer_id transaction_date online_order  \
0                   1           2         2950       25/02/2017        False   
1                   2           3         3120       21/05/2017         True   
2                   3          37          402       16/10/2017        False   
3                   4          88         3135       31/08/2017        False   
4                   5          78          787       01/10/2017         True   
...               ...         ...          ...              ...          ...   
19995           19996          51         1018       24/06/2017         True   
19996           19997          41          127       09/11/2017         True   
19997           19998          87         2284       14/04/2017         True   
19998           19999           6         2764       03/07/2017        False   
19999           20000          11         1144       22/09/2017         True   

      order_status           brand prod